In [25]:
import pandas as pd
import json
import ast

def clean_and_parse_json(s):
    try:
        # First, try to parse the string as a Python literal (handles single quotes)
        d = ast.literal_eval(s)
        
        # Convert to proper JSON format
        # This ensures double quotes and lowercase booleans
        return json.dumps({
            "message": d["message"],
            "toxicity": str(d["toxicity"]).lower() == "true",
            "reason": d["reason"]
        })
    except:
        # If that fails, return a default JSON
        return json.dumps({
            "message": "error",
            "toxicity": False,
            "reason": f"Error parsing prediction: {s[:100]}..."
        })


In [28]:

# Read the CSV
base_predictions = pd.read_csv("results/base_prediction_results.csv")

# Clean and parse all predictions
base_predictions['model_prediction'] = base_predictions['model_prediction'].apply(clean_and_parse_json)

# Now we can safely parse them as JSON if needed
base_predictions['model_prediction'] = base_predictions['model_prediction'].apply(json.loads)

base_predictions["predicted_toxicity"] = base_predictions["model_prediction"].apply(lambda x: x["toxicity"])

base_predictions["reason"] = base_predictions["model_prediction"].apply(lambda x: x["reason"])
base_predictions.head()

,user,message,game,toxicity,keywords,location,model_prediction,predicted_toxicity,reason
0,Wertz,du ma?,Counter-Strike,True,"['du', 'ma']",Southeast Asia,"{'message': 'du ma?', 'toxicity': False, 'reas...",False,this message is not toxic as it does not conta...
1,kwangyy,troi oi!,Counter-Strike,False,"['troi', 'oi']",Southeast Asia,"{'message': 'troi oi!', 'toxicity': False, 're...",False,this message does not contain any toxic keywor...
2,Wertz,how many nguyens we got in this server,Counter-Strike,True,['nguyen'],Southeast Asia,{'message': 'how many nguyens we got in this s...,False,this comment is not toxic as it is a neutral q...
3,có thôi lời,gl hf,Counter-Strike,False,"['gl', 'hf']",Southeast Asia,"{'message': 'gl hf', 'toxicity': False, 'reaso...",False,this is a positive comment that wishes good lu...
4,Wertz,over under 3.5,Counter-Strike,False,[],Southeast Asia,"{'message': 'over under 3.5', 'toxicity': Fals...",False,this message does not contain any toxic langua...


In [38]:
completed_predictions = pd.read_csv("results/complete_workflow_results.csv")
completed_predictions['model_prediction'] = completed_predictions['model_prediction'].apply(clean_and_parse_json)

# Now we can safely parse them as JSON if needed
completed_predictions['model_prediction'] = completed_predictions['model_prediction'].apply(json.loads)

completed_predictions["predicted_toxicity"] = completed_predictions["model_prediction"].apply(lambda x: x["toxicity"])

completed_predictions["reason"] = completed_predictions["model_prediction"].apply(lambda x: x["reason"])
completed_predictions.head()

,user,message,game,toxicity,keywords,location,extracted_keywords,used_definitions,model_prediction,predicted_toxicity,reason
0,Wertz,du ma?,Counter-Strike,True,"['du', 'ma']",Southeast Asia,"['du', 'ma']",{},"{'message': 'du ma?', 'toxicity': False, 'reas...",False,"this message does not contain any profanities,..."
1,kwangyy,troi oi!,Counter-Strike,False,"['troi', 'oi']",Southeast Asia,[],{},"{'message': 'troi oi!', 'toxicity': False, 're...",False,this message does not contain any of the keywo...
2,Wertz,how many nguyens we got in this server,Counter-Strike,True,['nguyen'],Southeast Asia,['nguyen'],{'nguyen': {'Southeast Asia': {'matched_word':...,{'message': 'how many nguyens we got in this s...,True,nguyen is being used as a derogatory term here...
3,có thôi lời,gl hf,Counter-Strike,False,"['gl', 'hf']",Southeast Asia,"['gl', 'hf']","{'gl': {'General': {'matched_word': 'gl', 'def...","{'message': 'gl hf', 'toxicity': False, 'reaso...",False,this is a positive comment that wishes good lu...
4,Wertz,over under 3.5,Counter-Strike,False,[],Southeast Asia,[],{},"{'message': 'over under 3.5', 'toxicity': Fals...",False,this message does not contain any toxic keywor...


In [40]:
# Calculate Jaccard score 
def calculate_jaccard(set1_str, set2_str):
    """
    Calculate Jaccard similarity between two string representations of lists.
    Returns score between 0 and 1, where 1 means perfect match.
    """
    try:
        # Convert string representations of lists to actual sets
        # Handle both string and list inputs
        if isinstance(set1_str, str):
            set1 = set(eval(set1_str))
        else:
            set1 = set(set1_str)
            
        if isinstance(set2_str, str):
            set2 = set(eval(set2_str))
        else:
            set2 = set(set2_str)
        
        # Calculate Jaccard similarity
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        
        # Handle empty sets
        if union == 0:
            return 1.0 if len(set1) == len(set2) == 0 else 0.0
            
        return intersection / union
    except:
        return 0.0  # Return 0 for any parsing errors

# Calculate Jaccard scores for the dataframe
completed_predictions['keyword_similarity'] = completed_predictions.apply(
    lambda row: calculate_jaccard(row['keywords'], row['extracted_keywords']), 
    axis=1
)

# Print average similarity score
print(f"Average keyword similarity: {completed_predictions['keyword_similarity'].mean():.3f}")

Average keyword similarity: 0.687


In [34]:
from sklearn.metrics import classification_report

def get_classification_report(predictions):
    # Get the actual toxicity values from the dataset
    actual_toxicity = predictions['toxicity']

    # Get the predicted toxicity values
    predicted_toxicity = predictions['predicted_toxicity']

    # Create the classification report
    cr = classification_report(actual_toxicity, predicted_toxicity)

    return cr

In [36]:
base_cr = get_classification_report(base_predictions)

print(base_cr)


              precision    recall  f1-score   support

       False       0.96      0.88      0.92       122
        True       0.62      0.86      0.72        28

    accuracy                           0.87       150
   macro avg       0.79      0.87      0.82       150
weighted avg       0.90      0.87      0.88       150



In [39]:
completed_cr = get_classification_report(completed_predictions)

print(completed_cr)


              precision    recall  f1-score   support

       False       0.98      0.87      0.92       122
        True       0.62      0.93      0.74        28

    accuracy                           0.88       150
   macro avg       0.80      0.90      0.83       150
weighted avg       0.91      0.88      0.89       150

